In [2]:
from logging import error, exception
# from httplib2.error import FailedToDecompressContent
import pandas as pd
import pygsheets
import os
from datetime import datetime, date
import sys

import gsheet_utility as gsheetUtility
import Email_Code as fkEmail
import data_processing as data_processing
import numpy as np

In [3]:
basePath = '/Users/a/Documents/GitHub/exception_visibility/'
date_master_file = pd.read_csv( '/Users/a/Documents/GitHub/exception_visibility/Dashboard/data/week_details.csv')
date_master_file['weekday'] = pd.to_datetime(date_master_file["weekday"],format = '%d/%m/%Y').dt.date
date_master_file['month_year'] = pd.to_datetime(date_master_file["month_year"],format = '%d/%m/%Y').dt.date

rc_raw_data_location = basePath + 'ETL/RC_Raw_Data/'
mh_raw_data_location = basePath + 'ETL/MH_Raw_Data/'
orphan_raw_data_location = basePath + 'ETL/Orphan_Raw_Data/'
hv_orphan_raw_data_location = basePath + 'ETL/HV_Orphan_Raw_Data/'

today = date.today()
today = str(today.year) + "-" + str('{:02d}'.format(today.month)) + "-" + str('{:02d}'.format(today.day))

rcgsheetdatapath = basePath + 'ETL/gsheet_data/rc_data/' + today + '.csv'
mhgsheetdatapath = basePath + 'ETL/gsheet_data/mh_data/' + today + '.csv'
orphangsheetdatapath = basePath + 'ETL/gsheet_data/orphan_data/' + today + '.csv'
hv_orphangsheetdatapath = basePath + 'ETL/gsheet_data/hv_orphan_data/' + today + '.csv'

error_message = ''
html_message = ''

In [4]:
html_message = html_message + "Data Ingestion Job for RC started at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S") + '<br>'
print(datetime.now().strftime("%m/%d/%Y %H:%M:%S") + ' Starting RC data process')
rcData = gsheetUtility.get_gsheet_data('https://docs.google.com/spreadsheets/d/1kCMwH7tq2RoiugdoDKe3xFYMgp_P4WvidwMtG3tNAUo/edit?usp=sharing', 'A3', rcgsheetdatapath,'RC')
# rcData = gsheetUtility.get_gsheet_data('https://docs.google.com/spreadsheets/d/1g4n416j5xGvgwV7KnDHpEPyN1mrsGgH5durMXT2hxHU/edit?usp=sharing','A3')
rcGsheetData = rcData.iloc[0:,0:19].copy()
rcDBColumns = ['rc_received_timestamp','received_by','rc_name','orphan_id','orphan_id_condition','fsn_identified_warehouse','wsn_id','final_area_rc','business_unit','supercategory','expiry_date','product_title','packaging_condition','damaged_physical_segregation','damaged_quantity','damaged_scan_box_id','non_damaged_physical_segregation','non_damaged_quantity','non_damaged_scan_box_id']
rcColumns= rcGsheetData.columns
rcGsheetData = gsheetUtility.assignDBColumns(rcGsheetData, gsheet_asset='Retrun Center', dbColumns=rcDBColumns)
rcGsheetData = rcGsheetData.fillna('NA')
rcGsheetData.orphan_id = rcGsheetData.orphan_id.str.upper()
rcGsheetData.orphan_id = rcGsheetData.orphan_id.apply(lambda x: 'NA' if ((x =='N/A') | (x == 'N\A')) else x)

html_message = html_message + "Fetching Data for RC completed at : " + datetime.now().strftime("%m/%d/%Y %H:%M:%S")+ '<br>'
# rcGsheetData = data_processing.func_column_strip(rcGsheetData)
rcGsheetData = data_processing.fun_replace_empty_data(rcGsheetData)

print(datetime.now().strftime("%d/%m/%Y %H:%M:%S") + ' Finishing fetching RC process Completed')
rcGsheetData = data_processing.datatype_conversion(rcGsheetData, 'rc_received_timestamp', 'datetime')
scanned_dates = rcGsheetData.rc_received_timestamp.unique()

print('coalesce columns')

rcGsheetData = data_processing.coalesce_columns(rcGsheetData, ['non_damaged_physical_segregation','damaged_physical_segregation'], 'physical_segregation')
rcGsheetData = data_processing.coalesce_columns(rcGsheetData, ['damaged_quantity','non_damaged_quantity'], 'quantity')
rcGsheetData = data_processing.coalesce_columns(rcGsheetData, ['damaged_scan_box_id','non_damaged_scan_box_id'], 'scan_box_id')
# print('droping unused columns')
# rcGsheetData.drop(columns=['non_damaged_physical_segregation','damaged_physical_segregation','damaged_quantity','non_damaged_quantity','damaged_scan_box_id','non_damaged_scan_box_id'], inplace=True)
rcGsheetData['scanned_date'] = pd.to_datetime(rcGsheetData['rc_received_timestamp']).dt.date
rcGsheetData = data_processing.fetch_date_details(rcGsheetData, 'rc_received_timestamp', date_master_file)
# print(rcGsheetData[['scanned_date','weekend','month_year','weeknum']])
created_raw_files = data_processing.fetch_created_files(rc_raw_data_location)
filenames = []
            
            

12/10/2021 08:43:55 Starting RC data process
12/10/2021, 08:43:55 Starting Fetching Gsheet Data
12/10/2021, 08:44:00 Fetching Gsheet Completed
RC
A3
10/12/2021 08:44:03 Finishing fetching RC process Completed
Converting rc_received_timestamp to datetime
Converted rc_received_timestamp to datetime
coalesce columns
mapping dates and weekend to scanned date
       scanned_date     weekend  month_year  weeknum
0        2021-08-13  14/08/2021  2021-08-01       33
1        2021-08-13  14/08/2021  2021-08-01       33
2        2021-08-13  14/08/2021  2021-08-01       33
3        2021-08-13  14/08/2021  2021-08-01       33
4        2021-08-13  14/08/2021  2021-08-01       33
...             ...         ...         ...      ...
131508   2021-12-09  11/12/2021  2021-12-01       50
131509   2021-12-09  11/12/2021  2021-12-01       50
131510   2021-12-09  11/12/2021  2021-12-01       50
131511   2021-12-09  11/12/2021  2021-12-01       50
131512   2021-12-09  11/12/2021  2021-12-01       50

[13151

In [11]:
print('creating raw files started')
filenames = data_processing.create_raw_files(rcGsheetData,'scanned_date',created_raw_files, rc_raw_data_location )
html_message = html_message + "<p> Raw files created for RC at : " + datetime.now().strftime("%d-%m-%Y %H:%M:%S") + "</p>" 
html_message = html_message + "<p> Total files created for RC : " + str(len(filenames)) + '<br>'
print('creating raw files completed')



creating raw files started
started creating raw files
raw file appended for 2021-08-13.csv
raw file appended for 2021-08-14.csv
raw file appended for 2021-08-15.csv
raw file appended for 2021-08-16.csv
raw file appended for 2021-08-17.csv
raw file appended for 2021-08-18.csv
raw file appended for 2021-08-19.csv
raw file appended for 2021-08-20.csv
raw file appended for 2021-08-21.csv
raw file appended for 2021-08-22.csv
raw file appended for 2021-08-23.csv
raw file appended for 2021-08-24.csv
raw file appended for 2021-08-25.csv
raw file appended for 2021-08-26.csv
raw file appended for 2021-08-27.csv
raw file appended for 2021-08-28.csv
raw file appended for 2021-08-29.csv
raw file appended for 2021-08-30.csv
raw file appended for 2021-08-31.csv
raw file appended for 2021-09-01.csv
raw file appended for 2021-09-02.csv
raw file appended for 2021-09-03.csv
raw file appended for 2021-09-04.csv
raw file appended for 2021-09-05.csv
raw file appended for 2021-09-06.csv
raw file appended for

In [16]:
created_raw_files = data_processing.fetch_created_files(rc_raw_data_location)
# print('Successfuly Completed')

rc_final_columns = ['rc_received_timestamp','received_by', 'rc_name', 'orphan_id', 'orphan_id_condition',
                    'fsn_identified_warehouse', 'wsn_id', 'final_area_rc', 'business_unit', 'supercategory', 'expiry_date', 
                    'product_title', 'packaging_condition', 'asset', 'physical_segregation', 'quantity', 'scan_box_id', 'scanned_date', 
                    'weekend', 'month', 'year','month_year','weeknum']

fetching names of already created raw files


In [17]:
# rc_dashboard_data = data_processing.collate_data_for_dashboard(datetime.now().date(), 180, created_raw_files, rc_raw_data_location, rc_final_columns)
# rc_dashboard_data.to_csv(basePath + 'Dashboard/data/rc_full_data.csv', index=False)
data_processing.collate_data_for_dashboard(datetime.now().date(), 180, created_raw_files, rc_raw_data_location, rc_final_columns)

,rc_received_timestamp,received_by,rc_name,orphan_id,orphan_id_condition,fsn_identified_warehouse,wsn_id,final_area_rc,business_unit,supercategory,...,asset,physical_segregation,quantity,scan_box_id,scanned_date,weekend,month,year,month_year,weeknum
0,2021-12-09 08:51:50,omprakash.y@flipkart.com,Binola,NaN,Not Scannable,No,NaN,NaN,BGM,Health & Beauty,...,Retrun Center,NaN,2,OLD7795,2021-12-09,11/12/2021,December,2021,2021-12-01,50
1,2021-12-09 08:52:08,omprakash.y@flipkart.com,Binola,NaN,Not Scannable,No,NaN,NaN,Lifestyle,Health & Beauty,...,Retrun Center,NaN,1,OLD7795,2021-12-09,11/12/2021,December,2021,2021-12-01,50
2,2021-12-09 08:52:29,omprakash.y@flipkart.com,Binola,NaN,Not Scannable,No,NaN,NaN,Lifestyle,Food products,...,Retrun Center,NaN,1,OLD7795,2021-12-09,11/12/2021,December,2021,2021-12-01,50
3,2021-12-09 08:52:50,omprakash.y@flipkart.com,Binola,NaN,Not Scannable,No,NaN,NaN,BGM,Other BGM,...,Retrun Center,NaN,2,OLD7795,2021-12-09,11/12/2021,December,2021,2021-12-01,50
4,2021-12-09 08:53:14,omprakash.y@flipkart.com,Binola,NaN,Not Scannable,No,NaN,NaN,Lifestyle,Health & Beauty,...,Retrun Center,NaN,4,OLD7795,2021-12-09,11/12/2021,December,2021,2021-12-01,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,2021-08-13 16:24:49,mukesh.dharam@flipkart.com,Binola,NaN,Missing,Yes,0CZ7NK_E,Store,NaN,NaN,...,Retrun Center,NaN,NaN,NaN,2021-08-13,14/08/2021,August,2021,2021-08-01,33
48,2021-08-13 16:28:21,mukesh.dharam@flipkart.com,Binola,NaN,Missing,Yes,XI4ZBC3,Returns Area,NaN,NaN,...,Retrun Center,NaN,NaN,NaN,2021-08-13,14/08/2021,August,2021,2021-08-01,33
49,2021-08-13 16:31:25,mukesh.dharam@flipkart.com,Binola,NaN,Missing,Yes,0ANCKQ_C,Store,NaN,NaN,...,Retrun Center,NaN,NaN,NaN,2021-08-13,14/08/2021,August,2021,2021-08-01,33
50,2021-08-13 16:38:57,mukesh.dharam@flipkart.com,Binola,NaN,Missing,Yes,0D1KEZ_V,Store,NaN,NaN,...,Retrun Center,NaN,NaN,NaN,2021-08-13,14/08/2021,August,2021,2021-08-01,33


In [14]:
created_raw_files

['2021-11-25.csv',
 '2021-11-19.csv',
 '2021-11-18.csv',
 '2021-11-24.csv',
 '2021-11-30.csv',
 '2021-11-26.csv',
 '2021-11-27.csv',
 '2021-11-23.csv',
 '2021-11-22.csv',
 '2021-08-29.csv',
 '2021-11-08.csv',
 '2021-11-20.csv',
 '2021-11-21.csv',
 '2021-11-09.csv',
 '2021-08-28.csv',
 '2021-09-27.csv',
 '2021-10-06.csv',
 '2021-10-12.csv',
 '2021-12-03.csv',
 '2021-12-02.csv',
 '2021-10-13.csv',
 '2021-10-07.csv',
 '2021-09-26.csv',
 '2021-09-18.csv',
 '2021-09-24.csv',
 '2021-09-30.csv',
 '2021-10-11.csv',
 '2021-10-05.csv',
 '2021-12-01.csv',
 '2021-10-04.csv',
 '2021-10-10.csv',
 '2021-09-25.csv',
 '2021-09-19.csv',
 '2021-09-21.csv',
 '2021-09-09.csv',
 '2021-10-14.csv',
 '2021-10-28.csv',
 '2021-12-05.csv',
 '2021-12-04.csv',
 '2021-10-29.csv',
 '2021-10-01.csv',
 '2021-10-15.csv',
 '2021-09-08.csv',
 '2021-09-20.csv',
 '2021-09-22.csv',
 '2021-10-03.csv',
 '2021-10-17.csv',
 '2021-12-06.csv',
 '2021-12-07.csv',
 '2021-10-16.csv',
 '2021-10-02.csv',
 '2021-09-23.csv',
 '2021-09-12